# Trying the LSTM model for caterogical prediction

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("your_preprocessed_data.csv")

# Split into features (X) and target (y)
X = df.drop('posture', axis=1).values
y = df['posture']

# Encode labels (posture classes)
le = LabelEncoder()
y = le.fit_transform(y)

# Reshape for LSTM (samples, time steps, features)
# Assuming each sample is a sequence of sensor readings over a fixed window
timesteps = 10  # Adjust this based on your data characteristics
X = X.reshape((X.shape[0], timesteps, X.shape[1]))

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, X.shape[2])))  # 64 LSTM units
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(len(le.classes_), activation='softmax'))  # Output layer for multi-class

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


# Visuals

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import seaborn as sns
import numpy as np


##  1. Training History: Loss and Accuracy


In [ ]:
# 1. Training History: Loss and Accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(model.history.history['loss'], label='Training Loss')
plt.plot(model.history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(model.history.history['accuracy'], label='Training Accuracy')
plt.plot(model.history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## 2. Confusion Matrix

In [ ]:
# 2. Confusion Matrix:
y_pred = np.argmax(model.predict(X_test), axis=-1)
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


## 3. Classification Report

In [ ]:
# 3. Classification Report:
print(classification_report(y_test, y_pred, target_names=le.classes_))


## 4. ROC Curve and AUC

In [ ]:
# 4. ROC Curve and AUC:
y_prob = model.predict(X_test)
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(le.classes_)):
    fpr[i], tpr[i], _ = roc_curve(y_test == i, y_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(10, 8))
for i in range(len(le.classes_)):
    plt.plot(fpr[i], tpr[i], label='ROC curve of class {0} (area = {1:0.2f})'
                                   ''.format(le.classes_[i], roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

## Example Predictions: (Assuming you have a new dataset 'new_X')

In [ ]:
# 5. Example Predictions: (Assuming you have a new dataset 'new_X')
new_X = ...  # Load/preprocess your new data
predictions = le.inverse_transform(np.argmax(model.predict(new_X), axis=-1))

print("Example predictions on new data:")
for i, prediction in enumerate(predictions[:10]):  # Show first 10 predictions
    print(f"Sample {i+1}: Predicted posture - {prediction}")
